In [165]:
import pandas as pd
import numpy as np
import os
import glob


### set directory

In [166]:
###set dir
input = 'Input\\'
output = 'Output\\'
mapping = 'mapping\\'
backup = 'Input\\Backup\\'
#os.mkdir(input)
#os.mkdir(output)
#os.mkdir(mapping)
#os.mkdir(backup)

### test before function

In [167]:
PATH = glob.glob("input/*.csv")[0]
df = pd.read_csv(PATH)

In [168]:
PATH = glob.glob("input/*.csv")[-1]
df = pd.read_csv(PATH)
df.rename(columns={"CUST_ID":"Customer Number"},inplace=True)
data = df.dropna(subset=['Customer Number'])
data.iloc[:,6]  = data.iloc[:,6].replace('2564','2565')

c:\Users\630039\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [169]:
PATH = glob.glob("mapping/*.xlsb")

In [189]:
for x in PATH:
    print(x[17:25])

20220131
20220228
20220331
20220430
20220531
20220630
20220731
20220831
20220930
20221031
20221130
20221231


### function

In [218]:
### import lastfile and initial clean data
def import_data():
    PATH = glob.glob("input/*.csv")[-1]
    df = pd.read_csv(PATH)
    df.rename(columns={"CUST_ID":"Customer Number"},inplace=True)
    #df = df[df['Customer Number'].str.contains(r'[0-9]')]
    df = df.dropna(subset=['Customer Number'])
    return df

### change data actual to review
def replace_date(df,actual,review):
    #data.iloc[:,6] = data.iloc[:,6].astype(str)
    df.iloc[:,6] = df.iloc[:,6].str.replace(actual,review)
    return df

### prepare mapping data
def mapping(df):
    list_mapping = glob.glob("mapping/*.xlsb")[0:2]
    for file in list_mapping:
     mapping = pd.read_excel(file,engine='pyxlsb',
                                  usecols=['Customer Number','Account Number','จำนวนวันที่ค้างชำระ','PF/NPF Flag']
                                  )
     mapping = mapping.sort_values(by='Customer Number', ascending=False)
     mapping = mapping.drop_duplicates(subset=['Customer Number'], keep ='first')
     mapping.set_index('Customer Number',inplace=True)
     print(f"Read CSV file mapping {file[17:25]} done!")
     mapping.rename(columns={"Account Number": "Account Number_{}".format(file[17:25]),"Customer Number": "Customer Number_{}".format(file[17:25]),"จำนวนวันที่ค้างชำระ": "DPD_{}".format(file[17:25]),
                             'PF/NPF Flag':'NPF_FLAG_{}'.format(file[17:25])},inplace = True)
     df = df.merge(mapping,on='Customer Number',how='left')
     print(f"mapping performance file {file} done!")
    return df


#for backup data
def backup_data(df,fname):
    df.to_csv(backup+fname,encoding='TIS-620',index=False)
    
#for export_csv data
def export_csv(df,fname):
    df.to_csv(output+fname,encoding='TIS-620',index=False)

In [215]:
df = import_data()
replace_date(df,'2564','2565')
df = mapping(df)

Read CSV file mapping 20220131 done!
mapping performance file mapping\RPISIC01_20220131_ข้อมูลรายตัวสำหรับตรวจสอบ (22 ประเภทธุรกิจ).xlsb done!
Read CSV file mapping 20220228 done!
mapping performance file mapping\RPISIC01_20220228_ข้อมูลรายตัวสำหรับตรวจสอบ (22 ประเภทธุรกิจ).xlsb done!


In [252]:
df

,ชื่อลูกค้า,ฝ่ายงาน,Customer Number,วัตุประสงค์ 2565,Grade 2565,Grade 2564,วันที่ประเมิน 2565,ประเภท 2565,Account Number_20220131,DPD_20220131,NPF_FLAG_20220131,Account Number_20220228,DPD_20220228,NPF_FLAG_20220228
0,นางมุทิตา มังคลัง และ นายสุรินทร์ มังคลัง,SMEs 1,610002906.0,Credit Review,sBBB,18.sB+,1/4/2565,S industries,5.879022e+09,0.0,1_PF,5.879022e+09,0.0,1_PF
1,หจก.พี.เอส.พี แมนชั่น,SMEs 1,710001224.0,Credit Review,18.sB+,18.sB+,30/5/2565,S industries,NaN,NaN,NaN,NaN,NaN,NaN
2,บจก.อมานะห์,SMEs 1,730008353.0,Credit Review,17.sBB,6.B+,20/1/2565,S Commerce,6.890101e+08,0.0,1_PF,6.890101e+08,0.0,1_PF
3,นายมนัส มากเลาะเลย์,SMEs 1,610000162.0,Credit Review,16.sBBB,16.sBBB,21/10/2565,S industries,NaN,NaN,NaN,NaN,NaN,NaN
4,นาย ธวัชชัย ยอดเวียน,SMEs 1,520000389.0,Credit Review,18.sB+,18.sB+,26/10/2565,S industries,5.876000e+09,0.0,1_PF,5.876000e+09,0.0,1_PF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,บ ทาคาว่า ออกแบบและก่อสร้าง,NaN,680006732.0,Credit Review,21.sC+,NaN,9/2/2565,S industries,5.390103e+08,0.0,1_PF,5.390058e+08,28.0,1_PF
1327,นายศิริพงษ์ สีฟอง น.ส.ตรีดาว ศิริศักดา,NaN,750745684.0,Credit Review,22.sDDD,NaN,12/7/2565,S industries,NaN,NaN,NaN,NaN,NaN,NaN
1328,นายวิรัตน์ นางมรกต บัวมหะกุล,NaN,750613908.0,Credit Review,19.sCCC,NaN,12/6/2565,S industries,NaN,NaN,NaN,NaN,NaN,NaN
1329,นางดารา นายเสรี มะลูลีม,NaN,750740671.0,Credit Review,21.sC+,NaN,14/12/2565,S industries,NaN,NaN,NaN,NaN,NaN,NaN


In [251]:
df.columns.str.contains(r'DPD')

array([False, False, False, False, False, False, False, False, False,
        True, False, False,  True, False])